In [ ]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install mysql-connector-python
%pip install plotly
%pip install requests

In [ ]:
import requests
import pandas as pd

all_products = []
for page_num in range(1, 130):  # 130 pages * 100 records/page = 12000 records
    url = f"https://world.openfoodfacts.org/api/v2/search?categories=chocolates&fields=code,product_name,brands,nutriments&page_size=100&page={page_num}"
    response = requests.get(url)
    data = response.json()
    all_products.extend(data['products'])

data = pd.DataFrame(all_products)


# Split nutriments
nutriments_df = pd.json_normalize(data['nutriments'])
data = pd.concat([data, nutriments_df], axis=1)

# selected the columns
selected_columns = [
    'brands',
    'code',
    'product_name',
    'carbohydrates_value',
    'energy-kcal_value',
    'energy-kcal_value_computed',
    'fat_value',
    'fiber_value',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g',
    'nova-group',
    'nutrition-score-fr',
    'proteins_value',
    'salt_value',
    'saturated-fat_value',
    'sodium_value',
    'sugars_value'
]

df_selected= data[selected_columns]

#Remove Nan value

df_selected = df_selected.dropna(subset = ["code"])

#normalize text columns (trim spaces, lowercase) and remove Nan value
df_selected["product_name"] = df_selected["product_name"].astype(str).str.strip().str.lower() #remove leading/trailing spaces
df_selected = df_selected[df_selected["product_name"] != '']   #remove empty strings
df_selected = df_selected.dropna(subset = ["product_name"]).reset_index(drop=True)

#normalize text columns (trim spaces, lowercase) and remove Nan value
df_selected["brands"] = df_selected["brands"].astype(str).str.strip().str.lower() #remove leading/trailing spaces
df_selected = df_selected[df_selected["brands"] != '']   #remove empty strings
df_selected = df_selected.dropna(subset = ["brands"]).reset_index(drop=True)

df_selected = df_selected.drop_duplicates().reset_index(drop=True)

df_selected.info()

df_selected.describe()

display(df_selected)

df_selected.to_excel("raw_chococrunch.xlsx", index=False)

In [10]:
# FULL CLEANED DATA PREPARATION CODE & FEATURE ENGINERING

import numpy as np

#Step-1 load dataset

df = pd.read_excel("raw_chococrunch.xlsx")

print("Intial shape:", df.shape)
print("initial missing value count:")
print(df.isnull().sum())

#Step-2 :Remove duplicates

if 'code' in df.columns:
    df = df.drop_duplicates(subset=['code'])
else :
    df = df.drop_duplicates(subset=['product_name','brands'])

df.reset_index(drop=True, inplace=True)
print("\n After removing duplicates:", df.shape)

#Step-4 : Handle Missing value(NaN)

num_cols = [
    'carbohydrates_value',
    'energy-kcal_value',
    'energy-kcal_value_computed',
    'fat_value',
    'fiber_value',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g',
    'nutrition-score-fr',
    'proteins_value',
    'salt_value',
    'saturated-fat_value',
    'sodium_value',
    'sugars_value'
]

cat_cols = ['brands','product_name']

#Fill numeric columns with median

for col in num_cols :
    if col in df.columns:
        df[col] = pd.to_numeric(df[col],errors='coerce') #ensure numeric
        median_val = df[col].median(skipna=True)
        df[col] = df[col].fillna(median_val)

#Fill categorical columns with mode

for col in cat_cols:
    if col in df.columns:
        mode_val = df[col].mode()[0] if not df[col].mode().empty else "unknown"
        df[col] = df[col].fillna(mode_val)

#Fill Nova group (categorical numeric)

if 'nova-group' in df.columns:
    df['nova-group'] = df['nova-group'].fillna(-1).astype(int)


#Step-5 Final Check

print("\n Final shape after cleaning:", df.shape)
print("\n Remaning missing values:")
print(df.isnull().sum()[df.isnull().sum()>0])

#Step-6 Outlier removal (Realistic + Statistical)

limits = {
    'carbohydrates_value' : 150,
    'energy-kcal_value' : 4000,
    'energy-kcal_value_computed' : 20000,
    'fat_value' : 80,
    'fiber_value' : 40,
    'fruits-vegetables-nuts-estimate-from-ingredients_100g' : 100,
    'proteins_value' : 100,
    'salt_value' : 10,
    'saturated-fat_value' : 50,
    'sodium_value' : 4,
    'sugars_value' : 100,}

# Filter
for col , max_val in limits.items() :
    if col in df.columns:
        before = df.shape[0]
        df = df[df[col] <= max_val]
        after = df.shape[0]
        print(f"{col}: Removed {before - after} rows(values > {max_val})")

#4. Reset index after cleaning
df = df.reset_index (drop=True)

#Step-6 Add new dervied columns

#1.sugar_to_carb_ratio
df['sugar_to_carb_ratio'] = df.apply(
    lambda x: x['sugars_value']/x['carbohydrates_value']
    if pd.notnull(x['sugars_value'])and pd.notnull(x['carbohydrates_value'])and x['carbohydrates_value'] !=0
    else 0,
    axis=1
    )
#2.calorie_category 
def calorie_cat(kcal):
    if kcal < 300:
        return "Low"
    elif kcal <= 450:
        return "Moderate"
    else:
        return "High"
    
df['calorie_category'] = df['energy-kcal_value'].apply(calorie_cat)

#3.sugar_category

def sugar_cat(sugar):
    if sugar < 10:
        return "Low sugar"
    elif sugar <=30:
        return "Moderate sugar"
    else :
        return "High sugar"
    
df['sugar_category']= df['sugars_value'].apply(sugar_cat)

#4.is_ultra_processed

df['is_ultra_processed']= df['nova-group'].apply(lambda x :"Yes" if x ==4 else "No")

display(df)

# Step-7 Verify & save
df.to_excel("Cleaned_chococrunch_with_points.xlsx", index=False)
print("\n File saved as 'Cleaned_chococrunch_with_points'")

Intial shape: (12523, 16)
initial missing value count:
brands                                                    577
code                                                        0
product_name                                              275
carbohydrates_value                                       756
energy-kcal_value                                         752
energy-kcal_value_computed                                849
fat_value                                                 724
fiber_value                                              3868
fruits-vegetables-nuts-estimate-from-ingredients_100g     705
nova-group                                               1236
nutrition-score-fr                                       1636
proteins_value                                            734
salt_value                                               1038
saturated-fat_value                                      1012
sodium_value                                             1038
sugars_value   

,brands,code,product_name,carbohydrates_value,energy-kcal_value,energy-kcal_value_computed,fat_value,fiber_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,nova-group,nutrition-score-fr,proteins_value,salt_value,saturated-fat_value,sodium_value,sugars_value,sugar_to_carb_ratio,calorie_category,sugar_category,is_ultra_processed
0,sidi ali,6111035000430,sidi ali,42.0,0.0,168.000,0.0,0.0,0.00000,-1,1.0,0.0,0.00000,0.00,0.000000,14.0,0.333333,Low,Moderate sugar,No
1,jaouda,6111242100992,perly,9.4,97.0,96.600,3.0,2.4,0.00000,3,7.0,8.0,0.37500,1.43,0.150000,4.1,0.436170,Low,Low sugar,No
2,aquafina,6111252421568,اكوافينا,0.0,0.0,0.000,0.0,0.0,0.00000,-1,0.0,0.0,0.00508,0.00,0.002032,0.0,0.000000,Low,Low sugar,No
3,jaouda,6111266962187,lait,4.9,45.0,45.500,1.5,2.4,0.00000,3,-3.0,3.1,0.22500,1.00,0.090000,0.0,0.000000,Low,Low sugar,No
4,milky food professional,6111246721261,fromage blanc nature,6.1,80.6,161.600,12.0,0.0,0.00000,3,-2.0,7.3,0.00000,0.00,0.000000,0.0,0.000000,Low,Low sugar,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11860,good life,5012869541803,spicy veg bean burgers,20.0,210.0,208.300,9.7,8.9,63.36575,3,-3.0,5.8,0.75000,0.60,0.300000,2.0,0.100000,Low,Low sugar,No
11861,al hamd,6111203004444,tartine la prairie classique 1kg,15.1,241.0,242.805,6.3,2.4,0.00000,4,7.0,5.6,0.37500,1.43,0.150000,4.1,0.271523,Low,Low sugar,Yes
11862,morrisons,5010251569190,baked beans in tomato sauce,13.8,90.0,80.400,0.8,2.4,77.00000,4,1.0,4.5,0.50000,0.20,0.200000,5.0,0.362319,Low,Low sugar,Yes
11863,the pantry,4088600086309,plain flour,80.9,381.0,380.600,1.4,4.0,0.00000,1,7.0,9.1,0.37500,0.40,0.150000,0.6,0.007417,Moderate,Low sugar,No



 File saved as 'Cleaned_chococrunch_with_points'


In [ ]:
#CHOCOCRUNCH ANALYTICS - visualization code & Explanation

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

sns.set_style("whitegrid")

#1.Load dataset

df = pd.read_excel("Cleaned_chococrunch_with_points.xlsx")

print("Dataset loaded successfully!")
print("Shape (rows,columns):", df.shape)
print("\n--column Names--")
print(list(df.columns))

In [ ]:
#1.Bar chart - Number of products per calorie / Sugar category

#Calorie
plt.figure(figsize=(8,5))
sns.countplot(x='calorie_category', hue='calorie_category', data=df, palette='Set1', order=['Low','Moderate','High'])
plt.title("Number of Products per Calorie category")
plt.xlabel("Calorie_category")
plt.ylabel("Count of Products")
plt.show()

#Sugar
plt.figure(figsize=(8,5))
sns.countplot(x='sugar_category', hue='sugar_category', data=df, palette='Set2', order=['Low sugar','Moderate sugar','High sugar'])
plt.title("Number of Products per Sugar category")
plt.xlabel("Sugar category")
plt.ylabel("Count of Products")
plt.show()

In [ ]:
#2.Pie Chart - Proportion of products by Nova Group

if 'nova-group' in df.columns:
    nova_counts = df['nova-group'].value_counts()
    plt.figure(figsize=(7,7))
    plt.pie(nova_counts,labels=nova_counts.index, autopct='%1.1f%%',startangle=90, colors=sns.color_palette("Set2"),shadow=True)
    plt.title("Proportion of Products by NOVA Group")
    plt.show()

In [ ]:
#3.Histograms-Distribution of Calories,Sugar

cols=['energy-kcal_value', 'sugars_value']
for col in cols:
    plt.figure(figsize=(8,5))
    sns.histplot(df[col], bins=40, kde=True)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
#4.Box Plot - Spread of Calories or Sugar Across Brands

top_brands = df['brands'].value_counts().nlargest(10).index
plt.figure(figsize=(12,8))
sns.boxplot(x='brands', y='energy-kcal_value', hue='brands', data=df[df['brands'].isin(top_brands)],palette='Pastel1')
plt.title("Calories Distribution Across Top 10 Brands")
plt.xlabel("Brands")
plt.ylabel("Energy kcal per 100g")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12,8))
sns.boxplot(x='brands', y='sugars_value', hue='brands', data=df[df['brands'].isin(top_brands)],palette='Set2')
plt.title("Sugar Distribution Across Top 10 Brands")
plt.xlabel("Brands")
plt.ylabel("Sugar g per 100g")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#5. Scatter Plot - Calories vs Sugars

plt.figure(figsize=(8,6))
sns.scatterplot(x='sugars_value', y='energy-kcal_value', data=df, hue='calorie_category', alpha=0.7)
plt.title("Calories vs Sugar Content")
plt.xlabel("Sugar g per 100g")
plt.ylabel("Energy kcal per 100g")
plt.show()

In [ ]:
#6. Hearmap - Correlation Between Nutritional Values

num_cols = [
    'energy-kcal_value','sugars_value',
    'carbohydrates_value','fat_value',
    'proteins_value','fiber_value',
    'salt_value','sodium_value',
    'sugar_to_carb_ratio']
plt.figure(figsize=(10,8))
sns.heatmap(df[num_cols].corr(), annot=True,cmap='coolwarm',fmt=".2f")
plt.title("Correlation Heatmap - Nutritional Values")
plt.show()

In [ ]:
#7.Top N Bar Plot - Brands with Highest Average calories

top_calorie_brands = df.groupby('brands')['energy-kcal_value'].mean().nlargest(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_calorie_brands.index, y=top_calorie_brands.values,hue= top_calorie_brands,palette='rocket')
plt.title("Top 10 Brands with Highest Average Calories")
plt.xlabel("Brands")
plt.ylabel("Average-Calories (Kcal per 100g)")
plt.xticks(rotation=45)
plt.show()